In [5]:
import requests
from bs4 import BeautifulSoup
import pymongo
import time

class Database(object):
    def __init__(self):
        self.connection = pymongo.MongoClient("localhost", 27017)
        self.db = self.connection["DSS4_SmallP"]
        self.collection = self.db["Movie_mojo"]
        
def Load_Data_mojo_yearly_dom(page,year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?page={}&view=releasedate&view2=domestic&yr={}&p=.htm'.format(page,year)
    return requests.get(Mojo_url)
def Load_Data_mojo_yearly_intl(year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?view2=worldwide&yr={}&p=.htm'.format(year)
    return requests.get(Mojo_url)

def Load_Data_mojo_movie_intl(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}&page=intl'.format(M_id)
    return requests.get(Mojo_url)



In [4]:
"""
Mojo 사이트의 연간 worldwide boxoffice를 2010년부터 2015년까지 정리해서 mongodb에 넘겨줌. 
(영화명, 모조 주소, domestic, oversea 매출 저장)
"""

dbs = Database()
for year in range(2010, 2016) :
    res = Load_Data_mojo_yearly_intl(year)
    content = res.content
    soup = BeautifulSoup(content,'html.parser')
    table = soup.find('table', attrs = {'cellpadding': '5'})
    trs = table.find_all('tr')
    for line in trs[2:-1] :
        arr_m = line.find_all('td')
        dict_m = {
            'Mname' : arr_m[1].get_text(), 
            'Domestic' : arr_m[4].get_text(), 
            'Oversea' : arr_m[6].get_text(),
            'Webpage' : arr_m[1].a.get('href'),
            'year' : year
        }
        dbs.collection.insert_one(dict_m)

#a = trs[3].find_all('td')
#print a[1]

NameError: name 'Database' is not defined

In [3]:
res = dbs.collection.find({"year": {"$eq":2015}})
for Movie in res:
    url =  u'http://www.boxofficemojo.com' + Movie['Webpage'] + '&page=intl'
    # example) http://www.boxofficemojo.com/movies/?id=starwars7.htm&page=intl


NameError: name 'dbs' is not defined

In [63]:
res = requests.get('http://www.boxofficemojo.com/movies/?id=starwars7.htm&page=intl')

In [11]:
dbs = Database()
dbq = dbs.collection.find({"year": {"$eq":2010},"budget" : { "$exists" : False }})
for tmp in dbq:
    print(tmp)
    M_id = tmp['Webpage']
    res = Load_Data_mojo_movie_intl(M_id)
    
    content = res.content
    soup = BeautifulSoup(content,'html.parser')
    
    table = soup.find('table', attrs = { "style" : "padding-top: 5px;"})
    tds = table.find_all('td')
    #<table border="0" cellspacing="1" cellpadding="3">
    table2 = soup.find('table', attrs = { "border" : "0", "cellspacing" : "1", "cellpadding" : "3"})
    tds2 = table2.find_all('td')
    
    for n, feature in enumerate(tds2):
        if feature.get_text() == 'South Korea':
            dict_m = {
                'rating_usa' : tds[8].b.get_text(),
                'release_usa' : tds[5].b.get_text(),
                'realease_kor' : tds2[n+2].get_text(),
                'gross_kor' : tds2[n+5].get_text(),
                'budget' : tds[9].b.get_text()
            }
            break
    else:
        print(tds[9])
        dict_m = {
            'rating_usa' : tds[8].b.get_text(),
            'release_usa' : tds[5].b.get_text(),
            'budget' : tds[9].b.get_text()
        }
    dict_m.update(tmp)
    dbs.collection.update_one({"_id":tmp["_id"]}, {"$set":dict_m})
    time.sleep(1)
    
#db.Movie_mojo.find({ "gross_kor" : { $exists : true } })

{u'Webpage': u'/movies/?id=wonderfulafterlife.htm', u'Mname': u"It's a Wonderful Afterlife", u'Domestic': u'n/a', u'year': 2010, u'_id': ObjectId('5897f5f927b9d95e44c05f62'), u'Oversea': u'$1.3'}
<td align="right"><script type="text/javascript">
    googletag.cmd.push(function() {
        googletag.defineSlot("/4215/imdb2.bom.movie", [[300, 250]], "ad-slot-2").addService(googletag.pubads().setTargeting("p", "rh"));
        googletag.pubads().enableSingleRequest();
        googletag.enableServices();
    });
    googletag.cmd.push(function() { googletag.display("ad-slot-2"); });
</script></td>


AttributeError: 'NoneType' object has no attribute 'get_text'

In [14]:
for x, i in enumerate(tds):
    print(x,i)

(0, <td align="center" valign="top">\n<a href="/movies/?page=media&amp;id=wonderfulafterlife.htm"><img border="1" src="https://images-na.ssl-images-amazon.com/images/M/MV5BMTg4NjgyNzE3NF5BMl5BanBnXkFtZTcwMjE4OTc3Mw@@._V1_UY222_CR0,0,150,222_AL.jpg" style="border-color: 000000"/></a> </td>)
(1, <td align="center" valign="top"><br><font face="Verdana" size="5"><b>It's a Wonderful Afterlife</b></font><br><br>\n<table border="0" cellpadding="0" cellspacing="0"><tr><td>\n<center><table bgcolor="#dcdcdc" border="0" cellpadding="4" cellspacing="1" width="95%"><tr bgcolor="#ffffff"><td valign="top">Distributor: <b><a href="/studio/chart/?studio=utv.htm">UTV Communications</a></b></td><td valign="top">Release Date: <b><nobr><a href="/schedule/?view=bydate&amp;release=theatrical&amp;date=2010-10-08&amp;p=.htm">October 8, 2010</a></nobr></b></td></tr><tr bgcolor="#ffffff"><td valign="top">Genre: <b>Foreign</b></td><td valign="top">Runtime: <b>1 hrs. 40 min.</b></td></tr><tr bgcolor="#ffffff"><td 